# Initialize repo

In [5]:
# download the MNIST dataset
import os
from torchvision.datasets import MNIST

MNIST_root = './data/db/MNIST'

# if not exists the path then create it
if not os.path.exists(MNIST_root):
    os.makedirs(MNIST_root)

# download the MNIST dataset
MNIST(MNIST_root, train=True, download=True)
MNIST(MNIST_root, train=False, download=True)

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data/db/MNIST
    Split: Test

# Continue Hopfield Layer

In [6]:
import torch
from torch import nn
import source.HopfieldLayer as HL
from torchvision import datasets
from torchvision import transforms
from torch.utils import data

# set transformer
transform=transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Grayscale(),
        transforms.Pad(2),  # pad to 32x32
        transforms.RandomAffine(
            degrees=5, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10
        ),
        transforms.ColorJitter(contrast=(0.9, 1.5)),
    ]
)

# load the training dataset
train_data = datasets.MNIST(root='./data/db/MNIST', train=True, download=False, transform=transform)
train_loader = data.DataLoader(train_data, batch_size=1_024, shuffle=True)


In [7]:
class CapacitorConv2d(torch.nn.Module):
    def __init__(
        self, in_channels: int, deep: int, encrease: int = 1, decrease: int = 1
    ):
        super(CapacitorConv2d, self).__init__()

        if encrease > 1 and decrease > 1:
            raise ValueError("encrease and decrease cannot be both greater than 1")
        if encrease < 1 or decrease < 1:
            raise ValueError("encrease and decrease must be not less than 1")
        if deep < 0:
            raise ValueError("deep must be greater than 0")

        out_channels = in_channels * 4 * encrease // decrease

        self.encode = torch.nn.Conv2d(
            in_channels, out_channels, (3, 3), padding=(1, 1)
        )
        self.list = torch.nn.ModuleList(
            [
                torch.nn.Sequential(
                    torch.nn.PReLU(out_channels),
                    torch.nn.Conv2d(
                        out_channels, out_channels, (3, 3), padding=(1, 1)
                    ),
                )
                for _ in range(deep)
            ]
        )
        self.bn = torch.nn.BatchNorm2d(out_channels)
        self.decode = torch.nn.MaxPool2d((2, 2), (2, 2))
        self.silu = torch.nn.SiLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.encode(x)
        for layer in self.list:
            x = layer(x)
        x = self.bn(x)
        x = self.decode(x)
        x = self.silu(x)
        return x

# Design model
class myModel(torch.nn.Module):
    def __init__(self):
        super(myModel, self).__init__()
        self.architecture = nn.ModuleList([
            # blocco convoluzionale
            CapacitorConv2d(1, 1, encrease=8),  # dimezza la dimensione e moltiplica per 4*8 il numero di canali
            CapacitorConv2d(32, 1, decrease=2),  # dimezza la dimensione e moltiplica per 4/2 il numero di canali
            CapacitorConv2d(64, 1, decrease=4),  # dimezza la dimensione e moltiplica per 4/4 il numero di canali

            # blocco Hopfield
            HL.HopfieldLayer(64, 8, 4*4, n_iter=1, temperature=0.5, classifier=True),
            nn.Flatten(),

            # blocco fully connected
            nn.Linear(64*8, 10),
        ])

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for layer in self.architecture:
            x = layer(x)
        return x

model = myModel()

# optimizer for the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
# train the Hopfield layer
from tqdm.notebook import tqdm

loss_fn = torch.nn.CrossEntropyLoss()
n_epochs = 20

progress_epoch = tqdm(range(n_epochs), desc="Epochs", leave=False)
for e in progress_epoch:
    
    images: torch.Tensor
    labels: torch.Tensor
    loss: float = 0.0
    
    progress_batch = tqdm(train_loader, desc="Batches", total=len(train_loader), leave=False)
    for images, labels in progress_batch:
        
        # flatten the image
        images = images.view(-1, 1, 32, 32)
        labels = labels.view(-1)

        # train the Hopfield layer
        optimizer.zero_grad()
        output: torch.Tensor = model(images)
        output = output.view(-1, 10)
        batch_loss: torch.Tensor = loss_fn(output, labels)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()

    loss /= len(train_loader)
    progress_epoch.set_postfix(
        loss=loss,
    )

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

In [10]:
# testing
test_data = datasets.MNIST(root='./data/db/MNIST', train=False, download=False, transform=transform)
test_loader = data.DataLoader(test_data, batch_size=1_024, shuffle=True)

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 1, 32, 32)
        labels = labels.view(-1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print("Accuracy: ", correct / total)

Accuracy:  0.9907
